
# Suivi de tempêtes dans les réanalyses ERA5 - Diagramme de phase

**Auteur : Frédéric FERRY - Ecole Nationale de la Météorologie (Météo France)**

Le diagramme de phase des cyclones (Cyclone Phase Space Diagram : moe.met.fsu.edu/~rhart/papers-hart/2003Hart.pdf) est un outil diagnostique utile permettant de caractériser la structure thermodynamique des cyclones tropicaux et extra-tropicaux. Il se base sur l'analyse de la symétrie thermique du système dans la basse troposphère et sur sa structure à cœur chaud ou à cœur froid dans la basse et la haute troposphère.

Les paramètres utilisés pour construire ce diagnostic permettent de décrire la structure à cœur chaud d'un cyclone tropical, la structure à cœur froid d'un cyclone extra-tropical ainsi que ses stades de développement (formation, intensification, occlusion, affaiblissement, développement de séclusions chaudes). Cet outil permet aussi d'analyser la structure hybride des cyclones subtropicaux et de suivre les transtions tropicales de cyclones subtropicaux ou transitions extra-tropicales de cyclones tropicaux.

Le présent calepin permet :
- Le suivi d'une tempête à partir de réanalyses ERA5 de Pmer et de géopotentiel et d'un fichier .txt de tracking obtenu grâce à un programme de tracking automatique (non fourni ici)
- Le calcul pas à pas des paramètres de symétrie thermique et de cœur chaud/cœur froid du diagrammme de phase
- Le tracé du diagramme de phase tel que disponible sur http://moe.met.fsu.edu/cyclonephase/

Le diagramme de phase est ici illustré avec le cas de la transition extra-tropicale du cyclone tropical Ophelia (octobre 2017)

Pour faire fonctionner ce calepin il est nécessaire de télécharger les données ERA5 au format netcdf, au pas de temps horaire et sur domaine limité (-100W-50E, 0-90N) :
- https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-single-levels?tab=form
- https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-pressure-levels?tab=form

In [ ]:
import os

import xarray as xr
import netCDF4

import math
import numpy as np
from scipy.ndimage import gaussian_filter

from cartopy import config
from cartopy.util import add_cyclic_point
import cartopy.feature as cfeature
import cartopy.crs as ccrs
from cartopy.mpl.geoaxes import GeoAxes
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import cartopy.io.img_tiles as cimgt

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import AxesGrid
import matplotlib.collections as collections

import IPython.display as IPdisplay, matplotlib.font_manager as fm
from PIL import Image
import glob

from sklearn.linear_model import LinearRegression

from tqdm import tqdm

In [ ]:
def plot_maxmin_points(lon, lat, data, extrema, nsize, symbol, color='k',
                       plotValue=True, transform=None):

    from scipy.ndimage import maximum_filter, minimum_filter

    if (extrema == 'max'):
        data_ext = maximum_filter(data, nsize, mode='nearest')
    elif (extrema == 'min'):
        data_ext = minimum_filter(data, nsize, mode='nearest')
    else:
        raise ValueError('Value for hilo must be either max or min')

    mxy, mxx = np.where(data_ext == data)

    for i in range(len(mxy)):
        #print('Low '+str(i+1))
        #print(lon[mxy[i], mxx[i]], lat[mxy[i], mxx[i]], int(data[mxy[i], mxx[i]]))
        ax.text(lon[mxy[i], mxx[i]], lat[mxy[i], mxx[i]], symbol, color=color, size=12,
                clip_on=True, horizontalalignment='center', verticalalignment='center',
                transform=transform)
        ax.text(lon[mxy[i], mxx[i]], lat[mxy[i], mxx[i]],
                '\n' + str(int(data[mxy[i], mxx[i]])),
                color=color, size=10, clip_on=True, fontweight='bold',
                horizontalalignment='center', verticalalignment='top', transform=transform)

In [ ]:
def make_animation(gif_filepath):
    from PIL import Image
    import os
    from IPython.display import Image as IPImage
    from IPython.display import display
    import time
    
    image_folder = './anim/'+storm+'/' # répertoire contenant les fichiers PNG
    output_file = gif_filepath # nom du fichier de sortie
    animation_speed = 0.9 # vitesse de l'animation en secondes
    
    # Liste tous les fichiers PNG dans le répertoire image_folder
    files = sorted(os.listdir(image_folder))
    image_files = [f for f in files if f.endswith('.png')]
    
    # Ouvre chaque fichier PNG et ajoute l'image à une liste
    images = []
    for filename in image_files:
        img = Image.open(os.path.join(image_folder, filename))
        images.append(img)
    
    # Crée l'animation GIF
    images[0].save(output_file, save_all=True, append_images=images[1:], duration=int(animation_speed*1000), loop=0)
    # Affiche l'animation GIF dans Jupyter
    with open(output_file,'rb') as f:
        display(IPImage(data=f.read(), format='png'))
    # Efface les fichiers PNG
    for filename in image_files:
        os.remove(image_folder+filename)

In [ ]:
def haversine(lon1, lat1, lon2, lat2):
    # convert decimal degrees to radians 
    lon1 = np.deg2rad(lon1)
    lon2 = np.deg2rad(lon2)
    lat1 = np.deg2rad(lat1)
    lat2 = np.deg2rad(lat2)

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a)) 
    r = 6371
    return c * r

In [ ]:
def get_bearing(lat1, lon1, lat2, lon2):
    dLon = (lon2 - lon1)
    x = math.cos(math.radians(lat2)) * math.sin(math.radians(dLon))
    y = math.cos(math.radians(lat1)) * math.sin(math.radians(lat2)) - math.sin(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.cos(math.radians(dLon))
    brng = np.arctan2(x,y)
    brng = np.degrees(brng)
    brng = (brng +360) % 360
    return brng

# Trajectoires de toutes les tempêtes

In [ ]:
storms=['Nov1982','Oct1987','Herta',
        'Viviane','Lothar','Martin',
        'Klaus','Xynthia','Joachim', 'Zeus',
        'Ophelia','Eleanor', 'Alex', 'Ciaran', 'Rolf', 'Zorbas']

In [ ]:
projection=ccrs.NearsidePerspective(central_longitude=0.0, central_latitude=45.0)
bounds = [(-60, 35, 30, 70)]
                             
fig = plt.figure(figsize=(15., 10.))
ax = fig.add_subplot(1, 1, 1, projection=projection)
ax.set_title('System tracks (ERA5 reanalysis)',loc='center',fontsize=14)
ax.gridlines(draw_labels=True, color='gray', alpha=0.8, linestyle='-')
#ax.stock_img()
LAND = cfeature.NaturalEarthFeature('physical', 'land', '10m',edgecolor='face',facecolor=cfeature.COLORS['land'],linewidth=.1)
ax.add_feature(LAND)
OCEAN = cfeature.NaturalEarthFeature('physical', 'ocean', '10m',edgecolor='face',facecolor=cfeature.COLORS['water'],linewidth=.1)
ax.add_feature(OCEAN)
ax.set_extent(*bounds, crs=ccrs.PlateCarree())

for storm in storms:
    liste_time,liste_lon,liste_lat,liste_pres=np.loadtxt('./txt/'+storm+'.txt',skiprows=0, dtype='U13,float,float,int',unpack=True)
    date1=str(liste_time[0])
    date2=str(liste_time[-1])
    ax.plot(liste_lon,liste_lat, transform=ccrs.PlateCarree(), label=storm+' '+date1+'-'+date2)
    ax.text(liste_lon[0]+1, liste_lat[0], storm, verticalalignment='center', horizontalalignment='right',transform=ccrs.PlateCarree())

ax.legend(loc="upper center", ncol=3)
plt.show()

figname='./figs/tracks2'
fig.savefig(figname+'.png')

# Suivi d'une tempête

In [ ]:
menu_options = {
    1: 'Option 1',
    2: 'Option 2',
    3: 'Option 3',
    4: 'Exit',
}

def print_menu():
    print ('1 -- Nov1982' )
    print ('2 -- Oct1987' )
    print ('3 -- Herta' )
    print ('4 -- Viviane' )
    print ('5 -- Lothar' )
    print ('6 -- Martin' )
    print ('7 -- Klaus' )
    print ('8 -- Xynthia' )
    print ('9 -- Joachim' )
    print ('10 -- Zeus' )
    print ('11 -- Ophelia' )
    print ('12 -- Eleanor' )
    print ('13 -- Alex' )
    print ('14 -- Ciaran' )    
    print ('15 -- Medicane Rolf' )
    print ('16 -- Medicane Zorbas (ERA5)' )
    print ('17 -- Medicane Zorbas (Analysis)' )
    
print_menu()

option = int(input('Enter number of the desired storm : ')) 
if option == 1:
    storm='Nov1982'
elif option == 2:
    storm='Oct1987'
elif option == 3:
    storm='Herta'
elif option == 4:
    storm='Viviane'
elif option == 5:
    storm='Lothar'
elif option == 6:
    storm='Martin'
elif option == 7:
    storm='Klaus'
elif option == 8:
    storm='Xynthia'
elif option == 9:
    storm='Joachim'
elif option == 10:
    storm='Zeus'
elif option == 11:
    storm='Ophelia'
elif option == 12:
    storm='Eleanor'
elif option == 13:
    storm='Alex'
elif option == 14:
    storm='Ciaran'
elif option == 15:
    storm='Rolf'
elif option == 16:
    storm='Zorbas'
elif option == 17:
    storm='Zorbas2'
else:
    print('Invalid option. Please enter a number between 1 and 12.')
    
if not os.path.exists('./anim/'+storm):
	os.mkdir('./anim/'+storm)

if not os.path.exists('./figs/'+storm):
	os.mkdir('./figs/'+storm)

if not os.path.exists('./figs/'+storm+'/MSL_tracking/'):
	os.mkdir('./figs/'+storm+'/MSL_tracking/')
    
dir_anim='./anim/'+storm+'/'

latS=30
latN=70
lonW=-60
lonE=35

if storm=='Rolf':
    latS=35
    latN=50
    lonW=0
    lonE=15
    
if storm=='Zorbas' or storm=='Zorbas2':
    latS=30
    latN=45
    lonW=5
    lonE=35

if storm=='Ciaran':
    latS=30
    latN=70
    lonW=-70
    lonE=35
    
bounds = [(lonW, lonE, latS, latN)]

In [ ]:
liste_time,liste_lon,liste_lat,liste_pres=np.loadtxt('./txt/'+storm+'.txt',
        skiprows=0, dtype='U13,float,float,int',unpack=True)
print(liste_time)

In [ ]:
interval=int(input("Entrer le pas de temps désiré : "))

liste_time=liste_time[::interval]
liste_lon=liste_lon[::interval]
liste_lat=liste_lat[::interval]
liste_pres=liste_pres[::interval]

print(liste_time)
print(liste_time.shape)

date1=str(liste_time[0])
date2=str(liste_time[-1])

In [ ]:
def plot_background(ax):
    ax.coastlines("10m", zorder=3, color='grey')
    ax.gridlines()
    ax.set_xticks(np.linspace(-180, 180, 37), crs=ccrs.PlateCarree())
    ax.set_yticks(np.linspace(-90, 90, 37), crs=ccrs.PlateCarree())
    lon_formatter = LongitudeFormatter(zero_direction_label=True)
    lat_formatter = LatitudeFormatter()
    ax.xaxis.set_major_formatter(lon_formatter)
    ax.yaxis.set_major_formatter(lat_formatter)
    return(ax)

projection=ccrs.PlateCarree()

vmin = np.min(liste_pres)
vmax = np.max(liste_pres)

fig = plt.figure(figsize=(15., 15.))
ax = fig.add_subplot(111, projection=projection)
ax.set_title(storm+' storm track from '+liste_time[0]+' to '+liste_time[-1],loc='center',fontsize=14)

#ax.stock_img()
LAND = cfeature.NaturalEarthFeature('physical', 'land', '10m',edgecolor='face',facecolor=cfeature.COLORS['land'],linewidth=.1)
ax.add_feature(LAND)
OCEAN = cfeature.NaturalEarthFeature('physical', 'ocean', '10m',edgecolor='face',facecolor=cfeature.COLORS['water'],linewidth=.1)
ax.add_feature(OCEAN)
# Create a Stamen terrain background instance.
#stamen_terrain = cimgt.Stamen('terrain-background')
# Add the Stamen data at zoom level 8.
#ax.add_image(stamen_terrain, 8)
plot_background(ax)
ax.set_extent(*bounds, crs=ccrs.PlateCarree())

ax.plot(liste_lon,liste_lat, color='k', transform=ccrs.PlateCarree())
for i in range(len(liste_time)):
    ax.scatter(liste_lon[i],liste_lat[i], c=liste_pres[i],
               cmap='plasma', vmin=vmin, vmax=vmax, s=50,
               label=liste_pres[i],transform=ccrs.PlateCarree(), zorder=10)

plt.legend(ncol=2)
plt.show()
figname='./figs/'+storm+'_era_track'
fig.savefig(figname+'.png')

fig = plt.figure(figsize=(15., 8.))
ax = fig.add_subplot(111)
ax.set_title(storm+' MSLP evolution from '+liste_time[0]+' to '+liste_time[-1],loc='center',fontsize=14)
ax.scatter(liste_time,liste_pres, color='red')
ax.plot(liste_time,liste_pres, color='red')

plt.xlabel('Date', fontsize=12)
plt.ylabel('Mean Sea Level Pressure (hPa)', fontsize=12)
locs, labels = plt.xticks()
plt.setp(labels, rotation=90)
plt.grid()
ax.tick_params(axis='x', labelsize=10)

plt.show()
figname='./figs/'+storm+'_era_slp_evolution'
fig.savefig(figname+'.png')

In [ ]:
f1    = xr.open_dataset("./data/"+storm+"/msl.nc")
f2    = xr.open_dataset("./data/"+storm+"/z.nc")
print(f1)
print(f2)

In [ ]:
interval=int(input("Enter the desired time interval : "))

In [ ]:
f1    = xr.open_dataset("./data/"+storm+"/msl.nc").sel(
    time=slice(date1,date2)).sel(
    latitude=slice(latN,latS)).sel(
    longitude=slice(lonW,lonE))

mslp = f1['msl'][::interval]/100
lat  = mslp.latitude.values
time  = mslp.time.values
lon  = mslp.longitude.values

print(mslp)

In [ ]:
f2    = xr.open_dataset("./data/"+storm+"/z.nc").sel(
    time=slice(date1,date2)).sel(
    latitude=slice(latN,latS)).sel(
    longitude=slice(lonW,lonE))

lev = f2.level.values

z = f2['z'][::interval]/9.81
print(z)

In [ ]:
lat  = mslp.latitude.values
lon  = mslp.longitude.values
lon_grid, lat_grid = np.meshgrid(lon, lat)

In [ ]:
print(lev)
lower_layer = int(input('Enter 1 for 900-600 hPa lower layer and 2 for 925-700 hPa lower layer : ')) 
if (lower_layer==1):
    listlev1=[900, 800, 700, 600]
if (lower_layer==2):
    listlev1=[925, 875, 825, 775, 700]
higher_layer = int(input('Enter 1 for 600-300 hPa higher layer and 2 for 700-400 hPa higher layer : ')) 
if (higher_layer==1):
    listlev2=[600, 500, 400, 300]
if (higher_layer==2):
    listlev2=[700, 600, 500, 400]

In [ ]:
print(listlev1)
print(z)
thickness=z.sel(level=listlev1[-1])-z.sel(level=listlev1[0])
print(thickness)
print(np.min(thickness))
print(np.max(thickness))

In [ ]:
mslp_levels = np.arange(900,1072,2)
#dz_levels = np.arange(3000,3525,25)
dz_levels=np.linspace(np.min(thickness),np.max(thickness),20)

for i in tqdm(range(len(time))):
    #print(str(time[i])[0:13])
    fig = plt.figure(figsize=(17., 12.))
    #fig.suptitle('Storm '+storm+' - Mean Sea level Pressure and tracking',fontsize=16)
    ax = fig.add_subplot(1, 1, 1, projection=projection)
    ax.set_title('Storm '+storm+' - MSLP tracking and low level thickness',loc='left',fontsize=14)
    ax.set_title(str(time[i])[0:13],loc='right',fontsize=14)
    plot_background(ax)
    ax.set_extent(*bounds, crs=ccrs.PlateCarree())
    
    # MSLP in contours and min max
    c1 = ax.contour(lon, lat, mslp[i,:,:], levels=mslp_levels, 
                    colors="black", linewidths=1, transform=ccrs.PlateCarree())
    ax.clabel(c1,fmt='%4.1i',fontsize=10)
    ax.scatter(liste_lon[i],liste_lat[i], c='green', transform=ccrs.PlateCarree())
    ax.plot(liste_lon[0:i+1],liste_lat[0:i+1], c='red', marker='+', transform=ccrs.PlateCarree())
    plot_maxmin_points(lon_grid, lat_grid, mslp[i,:,:], 'min', 25,
                       symbol='L', color='b', transform=ccrs.PlateCarree())
    cf = ax.contourf(lon, lat, thickness[i,:,:], levels=dz_levels, 
                     cmap='jet', transform=ccrs.PlateCarree())
    cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=0.5, pad=0.1)
    cb.set_label('m')
    #plt.show()
    figname='./anim/'+storm+'/MSL_thickness_tracking_'+str(time[i])[0:13]
    fig.savefig(figname+'.png',bbox_inches='tight')
    plt.close()

In [ ]:
gif_filepath = './anim/'+storm+'/MSL_thickness_tracking.gif'
make_animation(gif_filepath)

In [ ]:
max_dist = 500      # max distance in km

for i in tqdm(range(len(time))):
    #print(str(time[i])[0:13])
    data_slp = mslp[i,:,:]
    data_thickness = thickness[i,:,:]
        
    # Center coordinates
    clat = liste_lat[i]
    clon = liste_lon[i] 
        
    # Calculate distance between center and all other lat/lon pairs
    distance = haversine(lon[:,np.newaxis], lat, clon, clat)
    distance2=np.transpose(distance)
    # Mask distance array where distance > max_dist
    distance_m = np.ma.masked_greater(distance2, max_dist)
    # Mask the data array based on the distance mask
    data_slp_m = np.ma.masked_where(distance2 > max_dist, data_slp)
    data_thickness_m = np.ma.masked_where(distance2 > max_dist, data_thickness)

    fig = plt.figure(figsize=(17., 12.))
    ax = fig.add_subplot(1, 1, 1, projection=projection)
    ax.set_title('Storm '+storm+' - MSLP tracking and low level thickness',loc='left',fontsize=14)
    ax.set_title(str(time[i])[0:13],loc='right',fontsize=14)
    plot_background(ax)
    ax.set_extent(*bounds, crs=ccrs.PlateCarree())
    # MSLP in contours and min max
    c1 = ax.contour(lon, lat, data_slp_m, levels=mslp_levels,
                    colors="black", linewidths=1, transform=ccrs.PlateCarree())
    ax.clabel(c1,fmt='%4.1i',fontsize=10)
    ax.scatter(liste_lon[i],liste_lat[i], c='green', transform=ccrs.PlateCarree())
    ax.plot(liste_lon[0:i+1],liste_lat[0:i+1], c='red', marker='+', transform=ccrs.PlateCarree())
    cf = ax.contourf(lon, lat, data_thickness_m, levels=dz_levels, 
                     cmap='jet', transform=ccrs.PlateCarree())
    cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=0.5, pad=0.1)
    cb.set_label('m')
    #plt.show()
    figname='./anim/'+storm+'/MSL_tracking_zoom_'+str(time[i])[0:13]
    fig.savefig(figname+'.png',bbox_inches='tight')
    plt.close()

In [ ]:
gif_filepath = './anim/'+storm+'/MSL_tracking_zoom.gif'
make_animation(gif_filepath)

In [ ]:
max_dist = int(input('Enter the value of the maximum radius (in km) to compute the CPS diagrams : ')) 

# Calcul du paramètre de symétrie thermique du CPS

$$B=h*(\overline{Z_{600}-Z_{900}})_R-\overline{Z_{600}-Z_{900}})_L$$

h = hemisphere (1 = NH, -1 = SH).

$$(\overline{Z_{600}-Z_{900}})_R$$ : mean 900-600hPa thickness in semicircle right of motion

$$(\overline{Z_{600}-Z_{900}})_L$$ : mean 900-600hPa thickness in semicircle left of motion

In [ ]:
print('Available dates : ', liste_time[0:-1])

date = input('Enter a date (YYYY-MM-DDTXX) : ')
data = thickness.sel(time=date)
data_slp = mslp.sel(time=date)
print(data)

idate = list(liste_time).index(date)
print('index of date : ', idate)

In [ ]:
clat = liste_lat[idate]
clon = liste_lon[idate]
ang=get_bearing(clat, clon, liste_lat[idate+1], liste_lon[idate+1])

In [ ]:
qq_ang_all=np.zeros([len(lon),len(lat)],dtype='f')
for x in range(0,len(lon)):
    for y in range(0,len(lat)):
        qq_ang_all[x,y]=get_bearing(clat, clon, lat[y], lon[x])     
qq_ang_all=np.transpose(qq_ang_all)

In [ ]:
Zl=np.zeros([len(lat),len(lon)],dtype='f')
Zr=np.zeros([len(lat),len(lon)],dtype='f')

for jlat in tqdm(range(0,len(lat)-1)):
    for jlon in range(0,len(lon)-1):
        #set values along great circle line to missing (neither left nor right)
        if qq_ang_all[jlat,jlon] == ang:
            Zl[jlat,jlon] = 0
            Zr[jlat,jlon] = 0
        #for storm travel angles in quadrants 1 and 2 (NE and SE)
        elif (ang >= 0 and ang < 180):
            if (qq_ang_all[jlat,jlon] > ang and qq_ang_all[jlat,jlon]  < ang+180):
                Zl[jlat,jlon] = 0
                Zr[jlat,jlon] = data[jlat,jlon]
            else:
                Zr[jlat,jlon] = 0
                Zl[jlat,jlon] = data[jlat,jlon]
        #for storm travel angles in quadrants 3 and 4 (NW and SW)
        elif (ang >= 180 and ang < 360):
            if (qq_ang_all[jlat,jlon] > ang-180 and qq_ang_all[jlat,jlon] < ang):
                Zr[jlat,jlon] = 0
                Zl[jlat,jlon] = data[jlat,jlon]
            else:
                Zl[jlat,jlon] = 0
                Zr[jlat,jlon] = data[jlat,jlon]

In [ ]:
# Calculate distance between center and all other lat/lon pairs
distance = haversine(lon[:,np.newaxis], lat, clon, clat)
distance2=np.transpose(distance)
# Mask distance array where distance > max_dist
distance_m = np.ma.masked_greater(distance2, max_dist)
# Mask the data array based on the distance mask
data_slp_m = np.ma.masked_where(distance2 > max_dist, data_slp)
Zr_m = np.ma.masked_where(distance2 > max_dist, Zr)
Zl_m = np.ma.masked_where(distance2 > max_dist, Zl)

Zr_m[Zr_m == 0] = np.nan
Zl_m[Zl_m == 0] = np.nan

In [ ]:
levels=np.arange(0,370,10)

fig = plt.figure(figsize=(17., 12.))
ax = fig.add_subplot(1, 1, 1, projection=projection)
fig.suptitle('Storm '+storm+' - MSLP, low level thickness and right quadrant',fontsize=16)
ax.set_title(liste_time[idate],fontsize=14)
plot_background(ax)
ax.set_extent(*bounds, crs=ccrs.PlateCarree())
 
c0 = ax.contour(lon, lat, data_slp_m, mslp_levels,
                colors="black", linewidths=1, transform=ccrs.PlateCarree())
ax.clabel(c0,fmt='%4.1i',fontsize=10)
cf1 = ax.contourf(lon, lat, Zr_m, dz_levels,
                 cmap='jet', transform=ccrs.PlateCarree())
cb = fig.colorbar(cf1, orientation='horizontal', aspect=65, shrink=0.5, pad=0.1)
cb.set_label('m')
c3 = ax.contour(lon, lat, qq_ang_all, levels,
                colors="grey", linewidths=0.5, alpha=0.8, transform=ccrs.PlateCarree())
ax.clabel(c3,fmt='%4.1i',fontsize=10)

plt.show()

figname='./figs/'+storm+'/Thermal_symmetry0'
fig.savefig(figname+'.png',bbox_inches='tight')

In [ ]:
levels=np.arange(0,370,10)

fig = plt.figure(figsize=(17., 12.))
ax = fig.add_subplot(1, 1, 1, projection=projection)
fig.suptitle('Storm '+storm+' - MSLP, low level thickness and left quadrant',fontsize=16)
ax.set_title(liste_time[idate],fontsize=14)
plot_background(ax)
ax.set_extent(*bounds, crs=ccrs.PlateCarree())
 
c0 = ax.contour(lon, lat, data_slp_m, mslp_levels,
                colors="black", linewidths=1, transform=ccrs.PlateCarree())
ax.clabel(c0,fmt='%4.1i',fontsize=10)
cf2 = ax.contourf(lon, lat, Zl_m, dz_levels,
                 cmap='jet', transform=ccrs.PlateCarree())
cb = fig.colorbar(cf2, orientation='horizontal', aspect=65, shrink=0.5, pad=0.1)
cb.set_label('m')
c3 = ax.contour(lon, lat, qq_ang_all, levels,
                colors="grey", linewidths=0.5, alpha=0.8, transform=ccrs.PlateCarree())
ax.clabel(c3,fmt='%4.1i',fontsize=10)

plt.show()

figname='./figs/'+storm+'/Thermal_symmetry1'
fig.savefig(figname+'.png',bbox_inches='tight')

In [ ]:
# cyclone thermal symmetry parameter using classic average
Zr_mean=np.nanmean(Zr_m)
Zl_mean=np.nanmean(Zl_m)
B=Zr_mean-Zl_mean
print('Right quadrant thickness : ', Zr_mean)
print('Left quadrant thickness : ', Zl_mean)

if clat<0:
    B=-B

In [ ]:
levels=np.arange(0,370,10)

fig = plt.figure(figsize=(17., 12.))
ax = fig.add_subplot(1, 1, 1, projection=projection)
fig.suptitle('Storm '+storm+' - MSLP, low level thickness and right/left quadrants',fontsize=16)
ax.set_title(liste_time[idate]+' : B = '+str(int(B)),fontsize=14)
plot_background(ax)
ax.set_extent(*bounds, crs=ccrs.PlateCarree())
 
c0 = ax.contour(lon, lat, data_slp_m, mslp_levels,
                colors="black", linewidths=1, transform=ccrs.PlateCarree())
ax.clabel(c0,fmt='%4.1i',fontsize=10)
cf1 = ax.contourf(lon, lat, Zr_m, dz_levels,
                 cmap='jet', transform=ccrs.PlateCarree())
cf2 = ax.contourf(lon, lat, Zl_m, dz_levels,
                 cmap='jet', transform=ccrs.PlateCarree())
cb = fig.colorbar(cf1, orientation='horizontal', aspect=65, shrink=0.5, pad=0.1)
cb.set_label('m')
c3 = ax.contour(lon, lat, qq_ang_all, levels,
                colors="grey", linewidths=0.5, alpha=0.8, transform=ccrs.PlateCarree())
ax.clabel(c3,fmt='%4.1i',fontsize=10)

plt.show()

figname='./figs/'+storm+'/Thermal_symmetry'
fig.savefig(figname+'.png',bbox_inches='tight')

In [ ]:
B_series=[]

for i in tqdm(range(len(time)-1)):
    #print(liste_time[i])
    data = thickness[i,:,:]
    data_slp = mslp[i,:,:]

    # Center coordinates
    clat = liste_lat[i]
    clon = liste_lon[i]

    # Calculate bearing between center and all other lat/lon pairs
    qq_ang_all=np.zeros([len(lon),len(lat)],dtype='f')
    for x in range(0,len(lon)):
        for y in range(0,len(lat)):
            qq_ang_all[x,y]=get_bearing(clat, clon, lat[y], lon[x])     
    qq_ang_all=np.transpose(qq_ang_all)

    # Angle of travel of the center
    ang=get_bearing(clat, clon, liste_lat[i+1], liste_lon[i+1])

    # Define right/left cadrans
    Zl=np.zeros([len(lat),len(lon)],dtype='f')
    Zr=np.zeros([len(lat),len(lon)],dtype='f')

    for jlat in range(0,len(lat)-1):
        for jlon in range(0,len(lon)-1):
            #set values along great circle line to missing (neither left nor right)
            if qq_ang_all[jlat,jlon] == ang:
                Zl[jlat,jlon] = 0
                Zr[jlat,jlon] = 0
            #for storm travel angles in quadrants 1 and 2 (NE and SE)
            elif (ang >= 0 and ang < 180):
                if (qq_ang_all[jlat,jlon] > ang and qq_ang_all[jlat,jlon]  < ang+180):
                    Zl[jlat,jlon] = 0
                    Zr[jlat,jlon] = data[jlat,jlon]
                else:
                    Zr[jlat,jlon] = 0
                    Zl[jlat,jlon] = data[jlat,jlon]
            #for storm travel angles in quadrants 3 and 4 (NW and SW)
            elif (ang >= 180 and ang < 360):
                if (qq_ang_all[jlat,jlon] > ang-180 and qq_ang_all[jlat,jlon] < ang):
                    Zr[jlat,jlon] = 0
                    Zl[jlat,jlon] = data[jlat,jlon]
                else:
                    Zl[jlat,jlon] = 0
                    Zr[jlat,jlon] = data[jlat,jlon]

    # Calculate distance between center and all other lat/lon pairs
    distance = haversine(lon[:,np.newaxis], lat, clon, clat)
    distance2=np.transpose(distance)
    # Mask distance array where distance > max_dist
    distance_m = np.ma.masked_greater(distance2, max_dist)
    # Mask the data array based on the distance mask
    data_slp_m = np.ma.masked_where(distance2 > max_dist, data_slp)
    Zr_m = np.ma.masked_where(distance2 > max_dist, Zr)
    Zl_m = np.ma.masked_where(distance2 > max_dist, Zl)
    
    Zr_m[Zr_m == 0] = np.nan
    Zl_m[Zl_m == 0] = np.nan

    # cyclone thermal symmetry parameter using weighted average
    Zr_mean=np.nanmean(Zr_m)
    Zl_mean=np.nanmean(Zl_m)
    B=Zr_mean-Zl_mean
    
    if clat<0:
        B=-B
    #print('B = ',B)
    
    B_series.append(B)

    fig = plt.figure(figsize=(17., 12.))
    ax = fig.add_subplot(1, 1, 1, projection=projection)
    fig.suptitle('Storm '+storm+' - MSLP, low level thickness and right/left quadrants',fontsize=16)
    ax.set_title(liste_time[i]+' : B = '+str(int(B)),fontsize=14)
    plot_background(ax)
    ax.set_extent(*bounds, crs=ccrs.PlateCarree())

    c0 = ax.contour(lon, lat, data_slp_m, mslp_levels,
                    colors="black", linewidths=1, transform=ccrs.PlateCarree())
    ax.clabel(c0,fmt='%4.1i',fontsize=10)
    cf1 = ax.contourf(lon, lat, Zr_m, dz_levels,
                     cmap='jet', transform=ccrs.PlateCarree())
    cf2 = ax.contourf(lon, lat, Zl_m, dz_levels,
                     cmap='jet', transform=ccrs.PlateCarree())
    cb = fig.colorbar(cf1, orientation='horizontal', aspect=65, shrink=0.5, pad=0.1)
    cb.set_label('m')
    c3 = ax.contour(lon, lat, qq_ang_all, levels,
                    colors="grey", linewidths=0.5, alpha=0.8, transform=ccrs.PlateCarree())
    ax.clabel(c3,fmt='%4.1i',fontsize=10)

    #plt.show()
    figname='./anim/'+storm+'/Thermal_symmetry_'+liste_time[i]
    fig.savefig(figname+'.png',bbox_inches='tight')
    plt.close()

In [ ]:
gif_filepath = './anim/'+storm+'/Thermal_symmetry.gif'
make_animation(gif_filepath)

In [ ]:
print(B_series)

fig = plt.figure(figsize=(15., 8.))
ax = fig.add_subplot(111)
ax.set_title('Storm '+storm+' - low level thermal asymmetry parameter (B)'
             ,loc='left',fontsize=14)
ax.set_title(liste_time[0]+'-'+liste_time[-2],loc='right',fontsize=14)
ax.plot(time[0:-1], B_series, label='B')
plt.axhline(y=10, color='r')

figname='./figs/'+storm+'/Thermal_symmetry_timeseries'
fig.savefig(figname+'.png',bbox_inches='tight')

# Calcul du paramètre de coeur chaud/coeur froid du CPS

Approximation of the height gradient on an isobaric surface (i.e. magnitude of the geostrophic wind) :

$$\Delta{Z}=Z_{max}-Z_{min}$$

Magnitude of the cyclone thermal wind measured as the vertical profile of cyclone geopotential height gradient (cyclone strength) :

$$-V_T^L=\frac{\partial{\Delta Z}}{\partial{ln(P)}}_{900}^{600}$$

$$-V_T^U=\frac{\partial{\Delta Z}}{\partial{ln(P)}}_{600}^{300}$$

In [ ]:
print('Available dates : ', liste_time[0:-1])

date = input('Enter a date (YYYY-MM-DDTXX) : ')
data = thickness.sel(time=date)
data_slp = mslp.sel(time=date)
print(data)

idate = list(liste_time).index(date)
print('index of date : ', idate)

In [ ]:
clat = liste_lat[idate]
clon = liste_lon[idate]

deltaZ1=[]
deltaZ2=[]

#low layer loop
for plev in listlev1:

    ilev = list(lev).index(plev)
    data_z1 = z[idate,ilev,:,:]

    # Calculate distance between center and all other lat/lon pairs
    distance = haversine(lon[:,np.newaxis], lat, clon, clat)
    distance2=np.transpose(distance)
    # Mask distance array where distance > max_dist
    distance_m = np.ma.masked_greater(distance2, max_dist)
    # Mask the data array based on the distance mask
    data_z1_m = np.ma.masked_where(distance2 > max_dist, data_z1)
    zmin=np.min(data_z1_m)
    zmax=np.max(data_z1_m)
    delta_z1=zmax-zmin
    deltaZ1.append(int(delta_z1))

#high layer loop
for plev in listlev2:

    ilev = list(lev).index(plev)
    data_z2 = z[idate,ilev,:,:]

    # Calculate distance between center and all other lat/lon pairs
    distance = haversine(lon[:,np.newaxis], lat, clon, clat)
    distance2=np.transpose(distance)
    # Mask distance array where distance > max_dist
    distance_m = np.ma.masked_greater(distance2, max_dist)
    # Mask the data array based on the distance mask
    data_z2_m = np.ma.masked_where(distance2 > max_dist, data_z2)
    zmin=np.min(data_z2_m)
    zmax=np.max(data_z2_m)
    delta_z2=zmax-zmin
    deltaZ2.append(int(delta_z2))

In [ ]:
print(zmax)
print(zmin)

fig = plt.figure(figsize=(17., 12.))
ax = fig.add_subplot(1, 1, 1, projection=projection)
fig.suptitle('Storm '+storm+' - Geopotential at '+str(plev)+ 'hPa',fontsize=16)
ax.set_title(liste_time[idate]+ ' : zmax-zmin = '+str(int(delta_z2)),fontsize=14)
plot_background(ax)
ax.set_extent(*bounds, crs=ccrs.PlateCarree())
 
cf = ax.contourf(lon, lat, data_z2_m,
                     cmap='jet', transform=ccrs.PlateCarree())
cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=0.5, pad=0.1)
cb.set_label('m')

plt.show()

figname='./figs/'+storm+'/deltaz_map'
fig.savefig(figname+'.png',bbox_inches='tight')

In [ ]:
print("Lower layer vertical levels : ", listlev1)
print("Lower layer deltaZ : ", deltaZ1)
print("Higher layer vertical levels : ", listlev2)
print("Higher layer deltaZ : ", deltaZ2)

listlev11=[n * 100 for n in listlev1]
lnP1=np.log(listlev11)

listlev22=[n * 100 for n in listlev2]
lnP2=np.log(listlev22)

In [ ]:
VTL_series=[]
VTU_series=[]

for i in tqdm(range(len(time)-1)):
    #print(liste_time[i])

    # Center coordinates
    clat = liste_lat[i]
    clon = liste_lon[i]
    deltaZ1=[]
    deltaZ2=[]

    for plev in listlev1:

        ilev = list(lev).index(plev)
        data_z1 = z[i,ilev,:,:]
        # Calculate distance between center and all other lat/lon pairs
        distance = haversine(lon[:,np.newaxis], lat, clon, clat)
        distance2=np.transpose(distance)
        # Mask distance array where distance > max_dist
        distance_m = np.ma.masked_greater(distance2, max_dist)
        # Mask the data array based on the distance mask
        data_z1_m = np.ma.masked_where(distance2 > max_dist, data_z1)
        zmin=np.min(data_z1_m)
        zmax=np.max(data_z1_m)
        delta_z1=zmax-zmin
        deltaZ1.append(int(delta_z1))

    for plev in listlev2:

        ilev = list(lev).index(plev)
        data_z2 = z[i,ilev,:,:]

        # Calculate distance between center and all other lat/lon pairs
        distance = haversine(lon[:,np.newaxis], lat, clon, clat)
        distance2=np.transpose(distance)
        # Mask distance array where distance > max_dist
        distance_m = np.ma.masked_greater(distance2, max_dist)
        # Mask the data array based on the distance mask
        data_z2_m = np.ma.masked_where(distance2 > max_dist, data_z2)
        zmin=np.min(data_z2_m)
        zmax=np.max(data_z2_m)
        delta_z2=zmax-zmin
        deltaZ2.append(int(delta_z2))

    coef1=(deltaZ1[-1]-deltaZ1[0])/(lnP1[-1]-lnP1[0])
    VTL_series.append(int(coef1))

    coef2=(deltaZ2[-1]-deltaZ2[0])/(lnP2[-1]-lnP2[0])
    VTU_series.append(int(coef2))

In [ ]:
print(VTU_series)
print(VTL_series)

fig = plt.figure(figsize=(15., 8.))
ax = fig.add_subplot(111)
ax.set_title('Storm '+storm+' - Low level thermal wind parameter'
             ,loc='left',fontsize=14)
ax.set_title(liste_time[0]+'-'+liste_time[-2],loc='right',fontsize=14)
ax.plot(time[0:-1], VTL_series, label='-$VT_T^L$')

plt.axhline(y=0, color='r')

plt.show()

figname='./figs/'+storm+'/VTL_timeseries'
fig.savefig(figname+'.png',bbox_inches='tight')

fig = plt.figure(figsize=(15., 8.))
ax = fig.add_subplot(111)
ax.set_title('Storm '+storm+' - High level thermal wind parameter'
             ,loc='left',fontsize=14)
ax.set_title(liste_time[0]+'-'+liste_time[-2],loc='right',fontsize=14)
ax.plot(time[0:-1], VTU_series, label='-$VT_T^U$')

plt.axhline(y=0, color='r')

plt.show()

figname='./figs/'+storm+'/VTU_timeseries'
fig.savefig(figname+'.png',bbox_inches='tight')

In [ ]:
time_str=[x for x in range(len(time)-1)]
date_str=[x for x in range(len(time)-1)]
for i in range(len(time)-1):
    time_str[i] = str(time[i])
    date_str[i] = time_str[i][5:13]

################# CPS1 #################

if storm=='Zorbas':
    x1=-200
    x2=300
    y1=-20
    y2=60

else:
    x1=-600
    x2=300
    y1=-20
    y2=125

fig = plt.figure(figsize=(15., 8.))
ax = fig.add_subplot(111)
ax.set_title('Storm '+storm,loc='left',fontsize=14)
ax.set_title(liste_time[0]+'-'+liste_time[-2],loc='right',fontsize=14)
plt.axhline(y=10, color='grey', linewidth=10,zorder=1)
plt.axvline(x=0, color='grey', linewidth=10,zorder=1)
plt.plot(VTL_series, B_series)
plt.scatter(VTL_series,  B_series, c='k',zorder=2)
plt.scatter(VTL_series[0], B_series[0], c='green',zorder=2)
plt.scatter(VTL_series[-1], B_series[-1], c='red',zorder=2)
plt.xlabel("$-V_T^L$ [low level thermal wind]", fontsize = 16)
plt.ylabel("B [low level thickness asymmetry]", fontsize = 16)

plt.text(x1-35,-15,'Symmetric', rotation=90., color='red', fontsize = 14)
plt.text(x1-35, 20,'Asymmetric', rotation=90., color='red', fontsize = 14)
plt.text(-100, -30,'Cold core', rotation=0., color='blue', fontsize = 14)
plt.text(100, -30,'Warm core', rotation=0., color='red', fontsize = 14)
plt.xlim(x1, x2)
plt.ylim(y1, y2)
    
for i,type in enumerate(date_str):
    x = VTL_series[i]
    y = B_series[i]
    plt.text(x, y, type, fontsize=9)
    
xrange = [(-600, 600)]
yrange1 = (-20, 30)
c1 = collections.BrokenBarHCollection(xrange, yrange1, facecolor='blue', alpha=0.2)
ax.add_collection(c1)
xrange = [(-600, 600)]
yrange1 = (10, 130)
c1 = collections.BrokenBarHCollection(xrange, yrange1, facecolor='blue', alpha=0.2)
ax.add_collection(c1)
xrange = [(0, 300)]
yrange1 = (-20, 30)
c1 = collections.BrokenBarHCollection(xrange, yrange1, facecolor='red', alpha=0.2)
ax.add_collection(c1)
xrange = [(0, 300)]
yrange1 = (10, 130)
c1 = collections.BrokenBarHCollection(xrange, yrange1, facecolor='red', alpha=0.2)
ax.add_collection(c1)

plt.show()

figname='./figs/'+storm+'/CPS1'
fig.savefig(figname+'.png',bbox_inches='tight')

################# CPS2 #################

if storm=='Zorbas':
    x1=-300
    x2=300
    y1=-300
    y2=300

else:
    x1=-600
    x2=300
    y1=-600
    y2=300

fig = plt.figure(figsize=(15., 8.))
ax = fig.add_subplot(111)
ax.set_title('Storm '+storm,loc='left',fontsize=14)
ax.set_title(liste_time[0]+'-'+liste_time[-2],loc='right',fontsize=14)
plt.axhline(y=0, color='grey', linewidth=10,zorder=1)
plt.axvline(x=0, color='grey', linewidth=10,zorder=1)
plt.plot(VTL_series, VTU_series)
plt.scatter(VTL_series, VTU_series, c='k',zorder=2)
plt.scatter(VTL_series[0], VTU_series[0], c='green',zorder=2)
plt.scatter(VTL_series[-1], VTU_series[-1], c='red',zorder=2)

plt.xlabel("$-V_T^L$ [Low level thermal wind]", fontsize = 16)
plt.ylabel("$-V_T^U$ [High level thermal wind]", fontsize = 16)
plt.text(-200, y1+20,'Deep cold core', rotation=0., color='blue', fontsize = 14)
plt.text(90, 270,'Deep warm core', rotation=0., color='red', fontsize = 14)
plt.text(90, y1+20,'Shallow warm core', rotation=0., color='orange', fontsize = 14)

plt.xlim(x1, x2)
plt.ylim(y1, y2)

for i,type in enumerate(date_str):
    x = VTL_series[i]
    y = VTU_series[i]
    plt.text(x, y, type, fontsize=9)

xrange = [(0, 300)]
yrange1 = (0, 300)
c1 = collections.BrokenBarHCollection(xrange, yrange1, facecolor='r', alpha=0.2)
ax.add_collection(c1)
xrange = [(-600, 600)]
yrange1 = (-600, 600)
c1 = collections.BrokenBarHCollection(xrange, yrange1, facecolor='b', alpha=0.2)
ax.add_collection(c1)
xrange = [(-600, 600)]
yrange1 = (0, 300)
c1 = collections.BrokenBarHCollection(xrange, yrange1, facecolor='orange', alpha=0.2)
ax.add_collection(c1)
xrange = [(0, 300)]
yrange1 = (-600, 600)
c1 = collections.BrokenBarHCollection(xrange, yrange1, facecolor='orange', alpha=0.2)
ax.add_collection(c1)

plt.show()

figname='./figs/'+storm+'/CPS2'
fig.savefig(figname+'.png',bbox_inches='tight')